# Simple Monte Carlo Experiment

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

A handy routine to store the experiment results

In [ ]:
import time, gzip
import os, cPickle

def save(obj, path, prefix=None):
    prefix_ = "" if prefix is None else "%s_"%(prefix,)
    filename_ = os.path.join(path, "%s%s.gz"%(prefix_, time.strftime("%Y%m%d-%H%M%S"),))
    with gzip.open(filename_, "wb+", 9) as fout_:
        cPickle.dump(obj, fout_)
    return filename_

def load(filename):
    with gzip.open(filename, "rb") as f:
        return cPickle.load(f)

The path analyzer

In [ ]:
from crossing_tree import crossing_tree

def path_analyze(X, T, scale=1.0):
    xi, ti, offspring, Vnk, Znk, Wnk = crossing_tree(X, T, scale=scale, origin=X[0])
    # Sanity check.
    # for j in xrange(len(Znk)):
    #     assert np.allclose(2 * Vnk[j][:, :2].sum(axis=1) + 2, Znk[j])

    # Nn[n] -- the total number of crossings of grid with spacing \delta 2^n
    Nn = np.r_[len(xi), [len(index_) for index_ in offspring]] - 1

    # Dnk[n][k] -- the total number of crossings of grid \delta 2^{n+1}
    #  with exactly 2(k+1) subcrossings of grid \delta 2^n.
    freq = [np.bincount(Zk)[2::2] for Zk in Znk]
    Dnk = np.zeros((len(Znk), max(len(f) for f in freq)), np.int)
    for l, f in enumerate(freq):
        Dnk[l, :len(f)] = f

    # Vnde[n][d][e] -- the total number of up-down(e=0) and down-up(e=1)
    #  excursions in a downward (d=0) or upward (d=1) crossing of level
    #  n+1
    Vnde = np.array([(Vk[Vk[:, 2] < 0, :2].sum(axis=0),
                      Vk[Vk[:, 2] > 0, :2].sum(axis=0))
                     for Vk in Vnk], dtype=np.int)

    # Wnp[n][p] -- the p-th empirical quantile of the n-th level crossing
    #  durations.
    prc = [0.1, 0.5, 1.0, 2.5, 5.0, 10, 25, 50, 75, 90, 95, 97.5, 99, 99.5, 99.9]
    empty_ = np.full_like(prc, np.nan)
    Wnp = np.stack([np.percentile(Wk, prc) if len(Wk) > 0 else empty_ for Wk in Wnk])

    # The average crossing duration and its standard deviation
    Wavgn = np.array([np.mean(Wk) if len(Wk) > 0 else np.nan for Wk in Wnk])
    Wstdn = np.array([np.std(Wk) if len(Wk) > 0 else np.nan for Wk in Wnk])
    return scale, Nn, Dnk, Vnde, Wnp, Wavgn, Wstdn

Collect a list of results returned by path_analyze into aligned data tensors.

In [ ]:
def collect(results):
    results = list(results)

    scale_m = np.array([scale for scale, Nn, Dnk, Vnde, Wnp, Wavgn, Wstdn in results])

    Nmn = [Nn for scale, Nn, Dnk, Vnde, Wnp, Wavgn, Wstdn in results]
    L = max(Nn.shape[0] for Nn in Nmn)
    Nmn = np.stack([np.pad(Nn, (0, L - Nn.shape[0]), mode="constant").astype(np.float)
                    for Nn in Nmn])

    Dmnk = [Dnk for scale, Nn, Dnk, Vnde, Wnp, Wavgn, Wstdn in results]
    K = max(Dnk.shape[1] for Dnk in Dmnk)
    Dmnk = np.stack([np.pad(Dnk, ((0, L - 1 - Dnk.shape[0]), (0, K - Dnk.shape[1])),
                            mode="constant").astype(np.float)
                     for Dnk in Dmnk])

    Wmnp = np.stack([np.pad(Wnp.astype(np.float), ((0, L - 1 - Wnp.shape[0]), (0, 0)),
                            mode="constant", constant_values=np.nan)
                     for scale, Nn, Dnk, Vnde, Wnp, Wavgn, Wstdn in results])

    Vmnde = np.stack([np.pad(Vnde.astype(np.float), ((0, L - 1 - Vnde.shape[0]),
                                                     (0, 0), (0, 0)),
                             mode="constant", constant_values=np.nan)
                      for scale, Nn, Dnk, Vnde, Wnp, Wavgn, Wstdn in results])

    Wavgmn = np.stack([np.pad(Wavgn.astype(np.float), (0, L - 1 - Wavgn.shape[0]),
                              mode="constant", constant_values=np.nan)
                       for scale, Nn, Dnk, Vnde, Wnp, Wavgn, Wstdn in results])

    Wstdmn = np.stack([np.pad(Wstdn.astype(np.float), (0, L - 1 - Wstdn.shape[0]),
                              mode="constant", constant_values=np.nan)
                       for scale, Nn, Dnk, Vnde, Wnp, Wavgn, Wstdn in results])

    return scale_m, Nmn, Dmnk, Vmnde, Wmnp, Wavgmn, Wstdmn

A function implementing various delta choices.

In [ ]:
import warnings

def get_delta_method(delta=1.0):
    if isinstance(delta, str):
        if delta == "std":
            # the standard deviation of increments
            delta_ = lambda X: np.diff(X).std()
        elif delta == "med":
            # Use the median absolute difference [Jones, Rolls; 2009] p. 11 (arxiv:0911.5204v2)
            delta_ = lambda X: np.median(np.abs(np.diff(X)))
        elif delta == "iqr":
            # Interquartile range
            delta_ = lambda X: np.subtract(*np.percentile(np.diff(X), [75, 25]))
        elif delta == "rng":
            # Use the range estimate as suggested by Geoffrey on 2015-05-28
            warnings.warn("""Use of `range`-based grid resolution """
                          """is discouraged since it may cause misaligned """
                          """crossing trees.""", RuntimeWarning)
            delta_ = lambda X: (X.max() - X.min()) / (2**12)
        else:
            raise ValueError("""Invalid `delta` setting. Accepted values """
                             """are: [`iqr`, `std`, `med`, `rng`].""")
    elif isinstance(delta, float) and delta > 0:
        delta_ = lambda X: delta
    else:
        raise TypeError("""`delta` must be either a float, or a method """
                        """identifier.""")
    return delta_

An MC experiment kernel.

In [ ]:
from sklearn.base import clone

def mc_run(experiment_id, n_replications, generator, method):
    results = {method_: list() for method_ in method}
    deltas = [get_delta_method(method_) for method_ in method]

    generator = clone(generator)
    generator.start()
    for j in xrange(n_replications):
        T, X = generator.draw()
        for delta, method in zip(deltas, methods):
            results[method].append(path_analyze(X, T, scale=delta(X)))
    generator.finish()
    return experiment_id, results

## A mock up experiment

Setup the parallel backend

In [ ]:
from joblib import Parallel, delayed
par_ = Parallel(n_jobs=-1, verbose=10)

Initialize the random states

In [ ]:
random_state = np.random.RandomState(0xDEADC0DE)

# Create a bunch of random seed
MAX_RAND_SEED = np.iinfo(np.int32).max
seeds = random_state.randint(MAX_RAND_SEED, size=(8,))

Create the experiment schedule

In [ ]:
from crossing_tree.processes import FractionalBrownianMotion as FBM

N, H, methods = (1<<15) - 1, .65, ["med", "std", "iqr",]
jobs_ = (delayed(mc_run)(seed_, 125, FBM(N, hurst=H, random_state=seed_),
                         method=methods) for seed_ in seeds)

Run the experiment and flatten the results

In [ ]:
experiment_ids = list()
results_ = {method: list() for method in methods}
for id_, dict_ in par_(jobs_):
    experiment_ids.append(id_)
    for method in methods:
        results_[method].extend(dict_[method])

results = {key_: collect(list_) for key_, list_ in results_.iteritems()}

Inspect experiment IDs

In [ ]:
experiment_ids

Pick one to display

In [ ]:
scale_m, Nmn, Dmnk, Vmnde, Wmnp, Wavgmn, Wstdmn = results["std"]

Plot the number of crossings

In [ ]:
plt.plot(np.log(Nmn.T));
plt.show()

Compute the probability distribution of the number of sub-crossings.

In [ ]:
total_ = Dmnk.sum(axis=-1, keepdims=True)
total_[total_ < 1.] = 1.0
Dmnk /= total_

Plot the probabilities

In [ ]:
from math import log
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(111)
ax.plot(Dmnk.mean(axis=0).T)
ax.set_yscale("log", basey=2)

Plot the quantiles of the crossing durations

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(111)
prc = [0.1, 0.5, 1.0, 2.5, 5.0, 10, 25, 50, 75, 90, 95, 97.5, 99, 99.5, 99.9]
colors = plt.cm.rainbow_r(np.linspace(0, 1, num=len(Wmnp)), alpha=0.25)
for Wnp, Nn, col_ in zip(Wmnp, Nmn, colors):
#     wnp_ =  Wnp / 2**(np.arange(len(Nn)-1, dtype=np.float)[:, np.newaxis]/H)
    wnp_ =  Wnp
    for wp_ in wnp_[:-5]:
        ax.plot(wp_, prc, color=col_)
ax.set_xscale("log", basex=2)

Let's see how the base level resolution is distributed

In [ ]:
scale_m = {key_: res_[0] for key_, res_ in results.iteritems()}

for key_, res_ in results.iteritems():
    plt.hist(res_[0], bins=50, label=key_, lw=0, log=False);
plt.legend(loc="best", ncol=3)
plt.show()

## The fully felged experiment

This runs the experiment for the Fractional Brownian Motion process.

In [ ]:
random_state = np.random.RandomState(0xDEADC0DE)

MAX_RAND_SEED = np.iinfo(np.int32).max
seeds = random_state.randint(MAX_RAND_SEED, size=(8,))

par_ = Parallel(n_jobs=-1, verbose=0)
M, P, methods = 125, 23, ["med", "std", "iqr",]
for hurst in np.linspace(.5, .95, num=10):
    name_ = "FBM-%d-%0.2f-%dx%d"%(P, hurst, M, len(seeds))
    print name_,

    # Schedule the experiments
    schedule_ = (delayed(mc_run)(seed_, 125, FBM(N=(1 << P) - 1, hurst=hurst,
                                             random_state=seed_),
                             method=methods) for seed_ in seeds)

    # Run the experiment and collect the results
    tick_ = time.time()
    experiment_ids = list()
    results_ = {method: list() for method in methods}
    for id_, dict_ in par_(schedule_):
        experiment_ids.append(id_)
        for method in methods:
            results_[method].extend(dict_[method])
    results = {key_: collect(list_) for key_, list_ in results_.iteritems()}
    tock_ = time.time()
    
    filename_ = save((tick_, tock_, experiment_ids, results), "../results", name_)

    print "%0.3fsec."%(tock_ - tick_,), filename_